In [ ]:
import nibabel as nib
from nilearn import glm
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import h5py
import scipy
from scipy.io import loadmat 
from scipy.io import savemat
import os
import glob

In [ ]:
layername = ['/conv1','/conv2','/conv3','/conv4','/conv5','/fc6','/fc7','/fc8']
dataroot = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/feature_extracted/'

# CNN activation 
1. training movies

In [ ]:
lay = 0

for seg in range(1,19):
    print('seg:' , str(seg), '; Layer: ',layername[lay])
    secpath = (dataroot + 'AlexNet_feature_maps_processed_seg'+ str(seg) + '.h5')
    lay_feat = h5py.File(secpath, 'r')
    lay_feat = lay_feat[layername[lay] + '/data']
    lay_feat = lay_feat.value
    lay_feat = np.transpose(lay_feat,(3,2,1,0))
    print('lay_feat: ',lay_feat.shape)
    dim = lay_feat.shape
    print('dim:', dim)
    Nf = dim[-1]
    print('Nf:', Nf)
    if seg == 1:
        dim_array = np.asarray(dim)
        dim_array[-1] = Nf*18
        print(dim_array)
        lay_feat_concatenated = np.zeros(dim_array[0:4])
        print('lay_feat_concatenated:',lay_feat_concatenated.shape)
    lay_feat_concatenated[:,:,:,(seg-1)*Nf+0:seg*Nf] = lay_feat
    print(lay_feat.shape)

2. standarize time series

In [ ]:
dim = lay_feat_concatenated.shape
dim= np.asarray(dim)
lay_feat_concatenated = lay_feat_concatenated.reshape(np.prod(dim[:-1]),dim[-1],order='F').copy()
lay_feat_concatenated_mean = np.mean(lay_feat_concatenated,1)
lay_feat_concatenated_mean = np.expand_dims(lay_feat_concatenated_mean,-1)
lay_feat_concatenated_std = lay_feat_concatenated.std(1)
lay_feat_concatenated_std = np.expand_dims(lay_feat_concatenated_std,-1)
lay_feat_concatenated = np.subtract(lay_feat_concatenated,lay_feat_concatenated_mean)
lay_feat_concatenated = np.divide(lay_feat_concatenated,lay_feat_concatenated_std)
np.nan_to_num(lay_feat_concatenated, copy=False)
lay_feat_concatenated = lay_feat_concatenated.reshape(dim,order='F').copy()

In [ ]:
# scipy.io.savemat('AlexNet_feature_maps_process_layer1_concatenated.mat', lay_feat_concatenated, lay_feat_concatenated_mean, lay_feat_concatenated_std)


# fMRI

# Check correlation with .mat

1. lay_feat

In [ ]:
ori_lay_feat = loadmat('ori_lay_feat.mat')
print(ori_lay_feat.keys())
ori_lay_feat = ori_lay_feat['lay_feat']
print(ori_lay_feat.shape)

In [ ]:
lay_feat_list = []
for i in range(100):
    r = np.corrcoef(lay_feat[:,:,:,i].ravel(), ori_lay_feat[:,:,:,i].ravel())[0,1]
    lay_feat_list.append(r)
    
print(np.mean(lay_feat_list))

In [ ]:
plt.hist(lay_feat[:,:,:,i].ravel())

2. lay_feat_concatenated

In [ ]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/check_correaltion/lay_feat_concat_reshape.mat'
lay_concat_reshape = h5py.File(dir , 'r')
print(lay_concat_reshape.keys())
lay_concat_reshape = lay_concat_reshape['lay_feat_concatenated']
lay_concat_reshape = np.transpose(lay_concat_reshape,(1,0))
print(lay_concat_reshape.shape)

In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_concatenated[:,i].ravel(), lay_concat_reshape[:,i].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
print(np.array_equal(lay_feat_concatenated, lay_concat_reshape))

3. lay_feat_concatenated_mean

In [ ]:
lay_feat_concatenated_mean = np.mean(lay_feat_concatenated,1)
lay_feat_concatenated_mean = np.expand_dims(lay_feat_concatenated_mean,-1)
print(lay_feat_concatenated_mean.shape)

In [ ]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/check_correaltion/lay_feat_concat_mean.mat'
lay_concat_mean = h5py.File(dir , 'r')
print(lay_concat_mean.keys())
lay_concat_mean = lay_concat_mean['lay_feat_concatenated_mean']
lay_concat_mean = np.transpose(lay_concat_mean,(1,0))
print(lay_concat_mean.shape)

In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_concatenated_mean[i:,].ravel(), lay_concat_mean[i:,].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
print(np.array_equal(lay_feat_concatenated_mean, lay_concat_mean))

4. lay_feat_concatenated_std

In [ ]:
lay_feat_concatenated_std = lay_feat_concatenated.std(1)
lay_feat_concatenated_std = np.expand_dims(lay_feat_concatenated_std,-1)
print(lay_feat_concatenated_std.shape)

In [ ]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/check_correaltion/lay_feat_concat_std.mat'
lay_concat_std = h5py.File(dir , 'r')
print(lay_concat_std.keys())
lay_concat_std = lay_concat_std['lay_feat_concatenated_std']
lay_concat_std = np.transpose(lay_concat_std,(1,0))
print(lay_concat_std.shape)

In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_concatenated_std[i:,].ravel(), lay_concat_std[i:,].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
print(np.array_equal(lay_feat_concatenated_std, lay_concat_std))

5. lay_feat_concatenated bsxfun@minus

In [ ]:
lay_feat_concatenated = np.subtract(lay_feat_concatenated,lay_feat_concatenated_mean)
print(lay_feat_concatenated.shape)

In [ ]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/check_correaltion/lay_feat_minus.mat'
lay_concat_minus = h5py.File(dir , 'r')
print(lay_concat_minus.keys())
lay_concat_minus = lay_concat_minus['lay_feat_concatenated']
lay_concat_minus = np.transpose(lay_concat_minus,(1,0))
print(lay_concat_minus.shape)

In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_concatenated[:,i].ravel(), lay_concat_minus[:,i].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
print(np.array_equal(lay_feat_concatenated, lay_concat_minus))

6. lay_feat_concatenated bsxfun@rdivide

In [ ]:
lay_feat_concatenated = np.divide(lay_feat_concatenated,lay_feat_concatenated_std)
print(lay_feat_concatenated.shape)

In [ ]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/check_correaltion/lay_feat_divide.mat'
lay_concat_divide = h5py.File(dir , 'r')
print(lay_concat_divide.keys())
lay_concat_divide = lay_concat_divide['lay_feat_concatenated']
lay_concat_divide = np.transpose(lay_concat_divide,(1,0))
print(lay_concat_divide.shape)

In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_concatenated[:,i].ravel(), lay_concat_divide[:,i].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
print(np.array_equal(lay_feat_concatenated, lay_concat_divide))

7. lay_feat_concatenate isnan to zeros then reshape

In [ ]:
np.nan_to_num(lay_feat_concatenated, copy=False)
print(lay_feat_concatenated.shape)

In [ ]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/check_correaltion/lay_feat_nan.mat'
lay_nan = h5py.File(dir , 'r')
print(lay_nan.keys())
lay_nan = lay_nan['b']
print(lay_nan.shape)
lay_nan = np.transpose(lay_nan,(1,0))
print(lay_nan.shape)

In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_concatenated[:,i].ravel(), lay_nan[:,i].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
print(np.array_equal(lay_feat_concatenated, lay_nan))

In [ ]:
lay_feat_concatenated = lay_feat_concatenated.reshape(dim,order='F').copy()
print(lay_feat_concatenated.shape)

In [ ]:
dir = '/local_raid3/03_user/jungmin/02_data/Encoding_decoding/encoding_analyzing/02_Goal-Driven-DL/01_NeuralEncodingDecoding/sourcecode/source_code_v2/source_code_v2/check_correaltion/lay_feat_nan2reshape.mat'
lay_reshape_nan = h5py.File(dir , 'r')
print(lay_reshape_nan.keys())
lay_reshape_nan = lay_reshape_nan['lay_feat_concatenated']
print(lay_reshape_nan.shape)
lay_reshape_nan = np.transpose(lay_reshape_nan,(3,2,1,0))
print(lay_reshape_nan.shape)

In [ ]:
a_list = []
for i in range(100):
    r = np.corrcoef(lay_feat_concatenated[:,:,:,i].ravel(), lay_reshape_nan[:,:,:,i].ravel())[0,1]
    a_list.append(r)
    
print(np.mean(a_list))
print(np.array_equal(lay_feat_concatenated, lay_reshape_nan))